## User Input

In [1]:
data_file = r'./test_jupiter.ipynb'
config_file  = r'./ipynb2wp.json'

Do Not Change after this line. Update all required Configs in config file.

## Read data and config file

In [2]:
import json
import markdown
import re

In [3]:
def read_files(data_file, config_file):
    with open(data_file) as f:
        data = json.load(f)
    with open(config_file, 'r') as f:
        cfg = json.load(f)
    return [data, cfg]

## Functions

In [4]:
def ipynb_wp_md(source):
    data = ''.join(source)
    outval = markdown.markdown(data, output_format='html5')
    return outval

In [5]:
def ipynb_wp_code(code, cfg):
    updating_code = ''.join(code)
    mem = []
    #TODO : Add comment highlighter -json
    #TODO : Update colors to generic -json
    if bool(int(cfg['code']['color'])):
        for val in cfg['code']['priority']:
            if cfg['code'][val]['keyword_type']== 'list':
                keywords = [r'\b'+key+r'\b' for key in cfg['code'][val]['keywords']]
            else:
                key = '|'.join(cfg['code'][val]['keywords'])
                keywords = re.findall(key, updating_code)
            for key in keywords:
                if cfg['code'][val]['keyword_type']== 'list':
                    nkey = key.replace(r'\b', '')
                else:
                    nkey = key
                if val == 'operator':
                    key = '\\'+key
                updated_key = cfg['code'][val]['pre']+nkey+cfg['code'][val]['post']
                incr = len(nkey) - 1
                match = re.search(key, updating_code, re.S)
                while (match):
                    mem.append([match.start(), incr, updated_key])
                    updating_code = re.sub(key, " ", updating_code, 1)
                    match = re.search(key, updating_code, re.S)
        ins_array = list(updating_code)
        for data in mem[::-1]:
            ins_array[data[0]] = data[2]
            for itr in range(0, data[1]):
                ins_array.insert(data[0]+1, "")
        updating_code = "".join(ins_array)
    outdata = cfg['code']['block_pre']  + updating_code + cfg['code']['block_post'] 
    return outdata

In [6]:
def ipynb_wp_image(data, cfg):
    outdata = cfg['image']['block_pre']  + data + cfg['image']['block_post']
    return outdata

In [7]:
def ipynb_wp_performatted(data, cfg):
    for count, val in enumerate(data):
        if val.startswith("[<"):
            data[count] =""
    dval = "".join(data)
    if dval == "":
        outdata = ""
    else:
        outdata = cfg['performatted']['block_pre']  + dval + cfg['performatted']['block_post']
    return outdata    

In [8]:
def ipynb_wp_html(source, cfg):
    data = "".join(source)
    outval = ""
    key = cfg['html-table']['keywords']
    dval = re.findall(key, data, re.S)
    for datas in dval:
        if not datas.strip() == '':
            outval = outval + cfg['html-table']['block_pre'] + datas + cfg['html-table']['block_post']
    return outval

In [9]:
def ipynb2body(data_file, config_file):
    # Read  File
    [dataout, cfg] = read_files(data_file, config_file)
    outval = ''
    for i in range(0, len(dataout['cells'])):
        if dataout['cells'][i]['cell_type'] == 'markdown':
            outval = outval + '\n' + ipynb_wp_md(dataout['cells'][i]['source'])
        elif dataout['cells'][i]['cell_type'] == 'code':
            outval = outval + ipynb_wp_code(dataout['cells'][i]['source'], cfg)
            for opc in dataout['cells'][i]['outputs']:
                for key in opc['data']:
                    if 'image' in key:
                        outval = outval + ipynb_wp_image(opc['data'][key], cfg)
                    elif 'html' in key:
                        outval = outval + ipynb_wp_html(opc['data'][key], cfg)
                    else:
                        outval = outval + ipynb_wp_performatted(opc['data'][key], cfg)
                    break
        else:
            pass
    return outval             

In [10]:
html = ipynb2body(data_file, config_file)
with open("final.html", "w") as f:
    f.write(html)

In [10]:
data_file

NameError: name 'data_file' is not defined

In [245]:
cfg

{'code': {'color': '1',
  'custom_group1': {'keyword_type': 'list',
   'keywords': ['as'],
   'pre': "<span class='has-inline-color has-light-green-cyan-color'>",
   'post': '</span>'},
  'custom_group2': {'keyword_type': 'list',
   'keywords': ['for',
    'def',
    'in',
    'if',
    'elif',
    'else',
    'while',
    'return',
    'import',
    'not',
    'and',
    'or'],
   'pre': "<span class='has-inline-color has-vivid-green-cyan-color'><em>",
   'post': '</em></span>'},
  'function': {'keyword_type': 'regex',
   'keywords': ['\\s(.+?)\\('],
   'pre': "<span class='has-inline-color has-vivid-cyan-blue-color'>",
   'post': '</span>'},
  'operator': {'keyword_type': 'regex',
   'keywords': ['[=+-,*/<>]'],
   'pre': "<span class='has-inline-color has-vivid-red-color'>",
   'post': '</span>'},
  'digit': {'keyword_type': 'regex',
   'keywords': ['\\d+'],
   'pre': "<span class='has-inline-color has-luminous-vivid-amber-color'>",
   'post': '</span>'},
  'string': {'keyword_type':

In [231]:
m

'[=+-,*/<>]'

In [413]:
[dataout, cfg] = read_files(data_file, config_file)

In [414]:
dataout

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['## This is the main heading [H2]\n']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['*Lorem* ipsum dolor sit amet, **consectetur** adipiscing elit. Duis non hendrerit lectus, eu blandit massa. Sed ex tortor, cursus non dui sit amet, ullamcorper pulvinar mi. Nam accumsan est at tellus vestibulum, ut bibendum nibh ultrices. Nam varius bibendum turpis. Vivamus enim neque, consectetur non tempor vel, pellentesque et velit. Phasellus varius neque neque. Nulla quis erat sit amet nisl maximus tristique vel non velit. Integer sit amet pretium quam. Cras semper lectus a urna scelerisque, eget volutpat ante vestibulum. Etiam vel metus augue. Ut ultricies, metus eu ullamcorper ultrices, enim sapien viverra diam, sed imperdiet enim tellus at quam. Duis volutpat metus nec ultrices congue. Donec accumsan viverra ante, quis sodales justo sollicitudin ac. Integer dui mi, condimentum a lacus id, rhoncus sollicitudin 